# Universidad Autónoma de Nuevo León
## Facultad de Ciencias Fisico Matemáticas
### Maestría en Ciencia de Datos
#### Alumno: Francisco D. Treviño B.

### Tarea 2 (10 puntos). Operaciones con RDDs
### Obtener información de algún origen de datos (propio o de API)
### Convertir el origen de datos a RDD con pySpark
### Realizar alguna operación en el RDD, como estadísticas descriptivas básicas
### Subir la práctica a un repositorio público y etiquetarla claramente

# Instalación de Spark

## Se utilizó Docker para la instalación Spark usando una imagen, con el siguiente comando en Windows, según referencia en https://hub.docker.com/_/spark:
### docker pull spark


## Para correr Spark en Docker en modo interactivo se corre el siguiente comando:
### docker run -it spark:python3 /opt/spark/bin/pyspark

## Nota: no se utilizaron las siguientes instrucciones para instalar Spark.
#### !sudo apt update
#### !apt-get install openjdk-8-jdk-headless -qq > /dev/null
#### !wget -q https://archive.apache.org/dist/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
#### !tar xf spark-3.72.1-bin-hadoop3.2.tgz

In [ ]:
!pip install -q findspark

In [ ]:
!pip install pyspark

In [ ]:
!pip install py4j

In [ ]:
# Ejecutar el comando para iniciar Pyspark en un Notebook de Jupyter
# docker run -p 8888:8888 -p 4040:4040 -v "C:/Users/franc/Documents/UANL/MCD/07 Datos Masivos/Tareas/tarea-02":/home/jovyan/work --name pyspark-notebook -d 


In [1]:
import os
import sys

import findspark
findspark.init()
findspark.find()

import pyspark

from pyspark.sql import DataFrame, SparkSession
from typing import List
import pyspark.sql.types as T
import pyspark.sql.functions as F

### Dataframes de PySpark

In [2]:
# Inicializar sesión de Spark
from pyspark.sql import SparkSession

In [3]:
spark=SparkSession.builder.appName("Intraday_Data_Processing").getOrCreate()

# Configurar la sesión de Spark con más memoria y recursos
spark = SparkSession.builder \
    .appName("IntradayDataProcessing") \
    .config("spark.driver.memory", "4g") \
    .config("spark.executor.memory", "4g") \
    .config("spark.executor.cores", "2") \
    .getOrCreate()

In [4]:
# Paso 1: Convertir el DataFrame a RDD

# Lista de archivos CSV a cargar
csv_file_names = ["intraday_bmv_a.txt", "intraday_bmv_b.txt", "intraday_bmv_c.txt", "intraday_bmv_d.txt"]

# Cargar los archivos CSV, especificando que están delimitados por TAB
df_list = []
for csv_file_name in csv_file_names:
    df = spark.read.csv(csv_file_name, sep="\t", header=True, inferSchema=True)
    df_list.append(df)

# Unir todos los DataFrames en uno solo (si es necesario)
combined_df = df_list[0]
for df in df_list[1:]:
    combined_df = combined_df.union(df)

In [8]:
df.printSchema()

root
 |-- trade_time: timestamp (nullable = true)
 |-- match_number: integer (nullable = true)
 |-- exchange_instrument_id: integer (nullable = true)
 |-- system_timestamp: timestamp (nullable = true)
 |-- executed_quantity: integer (nullable = true)
 |-- executed_price: double (nullable = true)
 |-- amount: double (nullable = true)
 |-- buy_participant_id: integer (nullable = true)
 |-- buy_participant: string (nullable = true)
 |-- sell_participant_id: integer (nullable = true)
 |-- sell_participant: string (nullable = true)
 |-- auction_indicator: string (nullable = true)
 |-- settlement: integer (nullable = true)
 |-- operation_type: string (nullable = true)
 |-- concertation_type: string (nullable = true)
 |-- price_setter: integer (nullable = true)
 |-- lot: integer (nullable = true)
 |-- chart: integer (nullable = true)
 |-- symbol: string (nullable = true)



In [9]:
combined_df = combined_df.repartition(10)

In [10]:
# Convertir el DataFrame a un RDD
rdd = combined_df.rdd

In [11]:
# Mostrar el esquema del RDD (opcional)
rdd.take(5)  # Muestra los primeros 5 registros del RDD

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 11.0 failed 1 times, most recent failure: Lost task 0.0 in stage 11.0 (TID 69) (host.docker.internal executor driver): java.net.SocketException: Connection reset
	at java.net.SocketInputStream.read(Unknown Source)
	at java.net.SocketInputStream.read(Unknown Source)
	at java.io.BufferedInputStream.fill(Unknown Source)
	at java.io.BufferedInputStream.read(Unknown Source)
	at java.io.DataInputStream.readInt(Unknown Source)
	at org.apache.spark.api.python.PythonRunner$$anon$3.read(PythonRunner.scala:774)
	at org.apache.spark.api.python.PythonRunner$$anon$3.read(PythonRunner.scala:766)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.hasNext(PythonRunner.scala:525)
	at org.apache.spark.InterruptibleIterator.hasNext(InterruptibleIterator.scala:37)
	at scala.collection.Iterator.foreach(Itera

In [12]:
# Calcular estadísticas descriptivas básicas
count = executed_price_rdd.count()
mean = executed_price_rdd.mean()
min_value = executed_price_rdd.min()
max_value = executed_price_rdd.max()
stddev = executed_price_rdd.stdev()

# Mostrar los resultados
print(f"Count: {count}")
print(f"Mean: {mean}")
print(f"Min: {min_value}")
print(f"Max: {max_value}")
print(f"Standard Deviation: {stddev}")

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 4 in stage 13.0 failed 1 times, most recent failure: Lost task 4.0 in stage 13.0 (TID 74) (host.docker.internal executor driver): org.apache.spark.SparkException: Python worker exited unexpectedly (crashed)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator$$anonfun$1.applyOrElse(PythonRunner.scala:612)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator$$anonfun$1.applyOrElse(PythonRunner.scala:594)
	at scala.runtime.AbstractPartialFunction.apply(AbstractPartialFunction.scala:38)
	at org.apache.spark.api.python.PythonRunner$$anon$3.read(PythonRunner.scala:789)
	at org.apache.spark.api.python.PythonRunner$$anon$3.read(PythonRunner.scala:766)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.hasNext(PythonRunner.scala:525)
	at org.apache.spark.InterruptibleIterator.h

In [ ]:
# Summary stats
df.describe().show()